# Package

In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

In [7]:
path = 'input/store-sales-time-series-forecasting/'

# Feature Engineering

In [8]:
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

## Oil

In [9]:
df_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
df_oil['ma_oil'] = df_oil['dcoilwtico'].rolling(7).mean()
calendar = calendar.merge(df_oil, how='left', left_index=True, right_index=True)
calendar['ma_oil'].fillna(method='bfill', inplace=True)
calendar.head()

,dcoilwtico,ma_oil
2013-01-01,NaN,93.218571
2013-01-02,93.14,93.218571
2013-01-03,92.97,93.218571
2013-01-04,93.12,93.218571
2013-01-05,NaN,93.218571


## Day of week

In [10]:
calendar['dow'] = calendar.index.dayofweek
calendar.head()

,dcoilwtico,ma_oil,dow
2013-01-01,NaN,93.218571,1
2013-01-02,93.14,93.218571,2
2013-01-03,92.97,93.218571,3
2013-01-04,93.12,93.218571,4
2013-01-05,NaN,93.218571,5


## Hoilday

In [11]:
holidays = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True)
holidays = holidays.set_index('date').sort_index()
holidays = holidays[holidays.locale == 'National'] # National level only for simplicity
#holidays = holidays.groupby(holidays.index).first() # Keep one event only
holidays.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False


## Work Day

In [12]:
calendar['wd'] = True
calendar.loc[calendar.dow > 4, 'wd'] = False
calendar = calendar.merge(holidays, how='left', left_index=True, right_index=True)
calendar.loc[calendar.type == 'Bridge'  , 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'wd'] = True
calendar.head()

,dcoilwtico,ma_oil,dow,wd,type,locale,locale_name,description,transferred
2013-01-01,NaN,93.218571,1,False,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-02,93.14,93.218571,2,True,NaN,NaN,NaN,NaN,NaN
2013-01-03,92.97,93.218571,3,True,NaN,NaN,NaN,NaN,NaN
2013-01-04,93.12,93.218571,4,True,NaN,NaN,NaN,NaN,NaN
2013-01-05,NaN,93.218571,5,True,Work Day,National,Ecuador,Recupero puente Navidad,False


## Drop 12/25 which training set exclude

In [13]:
calendar = calendar.reset_index()
calendar.drop_duplicates(subset=['index'], keep='last', inplace=True, ignore_index=True)
calendar = calendar.set_index('index')
calendar = calendar.drop(index=['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'])
print(calendar.shape)

(1700, 9)


# Training

In [14]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()
df_train.head()

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0

## Label (y)

In [15]:
sdate = '2013-01-01'
edate = '2017-08-15'
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]
print(y.shape)
y.head()

(1684, 1782)


sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2013-01-01        0.0       0.0    0.0       0.0   0.0     0.000000   
2013-01-02        2.0       0.0    2.0    1091.0   0.0   470.652008   
2013-01-03        3.0       0.0    0.0     919.0   0.0   310.654999   
2013-01-04        3.0       0.0    3.0     953.0   0.0   198.365997   
2013-01-05        5.0       0.0    3.0    1160.0   0.0   301.057007   

                                                    ...                        \
store_nbr                                           ...         9               
family     CELEBRATION CLEANING  DAIRY        DELI  ... MAGAZINES       MEATS   
date                                                ...                         
2013-01-01         0.0      0.0    0.0    0.000000  ...       0.0    0.000000   
2013-01-02         0.0   1060.0  579.0  164.069000  ...       0.0  374.531006   
2013-01-03         0.0    836.0  453.0  151.582001  ...       0.0  400.863007   
2013-01-04         0.0    827.0  460.0  131.410995  ...       0.0  310.877991   
2013-01-05         0.0    811.0  464.0  118.612999  ...       0.0  447.684998   

                                                                           \
store_nbr                                                                   
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS     POULTRY   
date                                                                        
2013-01-01           0.0          0.0                     0.0    0.000000   
2013-01-02         482.0          0.0                     0.0  651.291992   
2013-01-03         372.0          0.0                     0.0  509.496002   
2013-01-04         324.0          0.0                     0.0  332.671997   
2013-01-05         461.0          0.0                     0.0  510.919006   

                                                                         
store_nbr                                                                
family     PREPARED FOODS PRODUCE SCHOOL AND OFFICE SUPPLIES    SEAFOOD  
date                                                                     
2013-01-01            0.0     0.0                        0.0   0.000000  
2013-01-02           83.0     0.0                        0.0  29.214001  
2013-01-03           66.0     0.0                        0.0  25.000000  
2013-01-04           57.0     0.0                        0.0  11.000000  
2013-01-05           84.0     0.0                        0.0  28.361000  

[5 rows x 1782 columns]

## Data point (X)

In [16]:
dp = DeterministicProcess(index = y.index, constant=False, order=1)
X = dp.in_sample()

# Extentions
X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['dow'] = calendar.loc[sdate:edate]['dow'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values
X['type'] = calendar.loc[sdate:edate]['type'].values
X = pd.get_dummies(X, columns=['dow'], drop_first=True)
X = pd.get_dummies(X, columns=['type'], drop_first=False)
print(X.shape)
X.head()


(1684, 15)


,trend,oil,wd,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day
date,,,,,,,,,,,,,,,
2013-01-01,1.0,93.218571,False,1,0,0,0,0,0,0,0,0,1,0,0
2013-01-02,2.0,93.218571,True,0,1,0,0,0,0,0,0,0,0,0,0
2013-01-03,3.0,93.218571,True,0,0,1,0,0,0,0,0,0,0,0,0
2013-01-04,4.0,93.218571,True,0,0,0,1,0,0,0,0,0,0,0,0
2013-01-05,5.0,93.218571,True,0,0,0,0,1,0,0,0,0,0,0,1


## Train

In [17]:
# model = LinearRegression()
# model.fit(X,y)
model_rf = RandomForestRegressor(n_estimators=250,random_state=2022, verbose=0)
model_rf.fit(X, y)

RandomForestRegressor(n_estimators=250, random_state=2022)

In [18]:
y_pred = pd.DataFrame(model_rf.predict(X), index=X.index, columns=y.columns)
y_pred=y_pred.stack(['store_nbr', 'family']).reset_index()
y_pred.head()

,date,store_nbr,family,sales
0,2013-01-01,1,AUTOMOTIVE,0.196
1,2013-01-01,1,BABY CARE,0.000
2,2013-01-01,1,BEAUTY,0.284
3,2013-01-01,1,BEVERAGES,179.992
4,2013-01-01,1,BOOKS,0.000


In [19]:
y_target=y.stack(['store_nbr', 'family']).reset_index().copy()
y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0
y_target.groupby('family').apply(lambda x: np.sqrt(mean_squared_log_error(x['sales'], x['sales_pred'])))

family
AUTOMOTIVE                    0.267844
BABY CARE                     0.089108
BEAUTY                        0.232869
BEVERAGES                     0.480921
BOOKS                         0.059981
BREAD/BAKERY                  0.356189
CELEBRATION                   0.257322
CLEANING                      0.429009
DAIRY                         0.378649
DELI                          0.311256
EGGS                          0.294269
FROZEN FOODS                  0.301998
GROCERY I                     0.535410
GROCERY II                    0.298225
HARDWARE                      0.240249
HOME AND KITCHEN I            0.329849
HOME AND KITCHEN II           0.254304
HOME APPLIANCES               0.158854
HOME CARE                     0.569891
LADIESWEAR                    0.241072
LAWN AND GARDEN               0.260911
LINGERIE                      0.318906
LIQUOR,WINE,BEER              0.649091
MAGAZINES                     0.209287
MEATS                         0.316664
PERSONAL CARE     

# Prediction

In [20]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()
df_test['sales'] = 0
df_test.head()

sales
store_nbr family     date             
1         AUTOMOTIVE 2017-08-16      0
                     2017-08-17      0
                     2017-08-18      0
                     2017-08-19      0
                     2017-08-20      0

## Label (y_test)

In [21]:
stest = '2017-08-16'
etest = '2017-08-31'
y_test = df_test.unstack(['store_nbr', 'family']).loc[stest:etest]
print(y_test.shape)
y_test.head()

(16, 1782)


sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-08-16          0         0      0         0     0            0   
2017-08-17          0         0      0         0     0            0   
2017-08-18          0         0      0         0     0            0   
2017-08-19          0         0      0         0     0            0   
2017-08-20          0         0      0         0     0            0   

                                            ...                                \
store_nbr                                   ...         9                       
family     CELEBRATION CLEANING DAIRY DELI  ... MAGAZINES MEATS PERSONAL CARE   
date                                        ...                                 
2017-08-16           0        0     0    0  ...         0     0             0   
2017-08-17           0        0     0    0  ...         0     0             0   
2017-08-18           0        0     0    0  ...         0     0             0   
2017-08-19           0        0     0    0  ...         0     0             0   
2017-08-20           0        0     0    0  ...         0     0             0   

                                                                        \
store_nbr                                                                
family     PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY PREPARED FOODS   
date                                                                     
2017-08-16            0                       0       0              0   
2017-08-17            0                       0       0              0   
2017-08-18            0                       0       0              0   
2017-08-19            0                       0       0              0   
2017-08-20            0                       0       0              0   

                                                       
store_nbr                                              
family     PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                   
2017-08-16       0                          0       0  
2017-08-17       0                          0       0  
2017-08-18       0                          0       0  
2017-08-19       0                          0       0  
2017-08-20       0                          0       0  

[5 rows x 1782 columns]

## Data point (X_test)

In [22]:
X_test = dp.out_of_sample(steps=16)

# Extentions
X_test['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test['dow'] = calendar.loc[stest:etest]['dow'].values
X_test['wd']   = calendar.loc[stest:etest]['wd'].values
X_test['type'] = calendar.loc[stest:etest]['type'].values
X_test = pd.get_dummies(X_test, columns=['dow'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['type'], drop_first=False)

# No national level events in this period
X_test[['type_Additional', 'type_Event', 'type_Holiday', 'type_Transfer', 'type_Bridge', 'type_Work Day']] = 0
X_test.index.names = ['date']
print(X_test.shape)
X_test.head()

(16, 15)


,trend,oil,wd,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,type_Additional,type_Event,type_Holiday,type_Transfer,type_Bridge,type_Work Day
date,,,,,,,,,,,,,,,
2017-08-16,1685.0,48.281429,True,0,1,0,0,0,0,0,0,0,0,0,0
2017-08-17,1686.0,47.995714,True,0,0,1,0,0,0,0,0,0,0,0,0
2017-08-18,1687.0,47.852857,True,0,0,0,1,0,0,0,0,0,0,0,0
2017-08-19,1688.0,47.688571,False,0,0,0,0,1,0,0,0,0,0,0,0
2017-08-20,1689.0,47.688571,False,0,0,0,0,0,1,0,0,0,0,0,0


## Predict

In [23]:
sales_pred = pd.DataFrame(model_rf.predict(X_test), index=X_test.index, columns=y_test.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family']).reset_index()
sales_pred['sales'] = sales_pred['sales'].clip(0.) # Sales should be >= 0
sales_pred

d:\Python\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,date,store_nbr,family,sales
0,2017-08-16,1,AUTOMOTIVE,5.068000
1,2017-08-16,1,BABY CARE,0.000000
2,2017-08-16,1,BEAUTY,3.436000
3,2017-08-16,1,BEVERAGES,2467.928000
4,2017-08-16,1,BOOKS,0.260000
...,...,...,...,...
28507,2017-08-31,9,POULTRY,336.605635
28508,2017-08-31,9,PREPARED FOODS,92.603887
28509,2017-08-31,9,PRODUCE,1170.038457
28510,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,58.380000


In [24]:
df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = sales_pred['sales'].values
df_sub.to_csv('submission.csv', index=True)